 # 数据加载,存储与文件格式
 NumPy提供了一个低价但异常高效的二进制数据加载和存储机制,包括对内存映射数组的支持.
 输入输出通常可以划分为几个大类:读取文本文件和其他更高效的磁盘存储格式,加载数据库中的数据,利用Web API操作网络资源.
 ## 读写文本格式的数据
 pandas提供了一些用于将表格型数据读取为DataFrame对象的函数.其中read_csv和read_table使用率最高.
 这些函数可以划分为以下几大类:
 - 索引:将一个或多个列当作返回的DataFrame处理,以及是否从文件,用户获取列名.
 - 类型推断和数据转换:包括用户定义值的转换,缺失值标记列表等.
 - 日期解析:包括组合功能,比如将分散在多个列中的日期时间信息组合成结果中的单个列.
 - 迭代:支持对大文件进行逐块迭代.
 - 不规整数据问题:跳过一些行,页脚,注释或其他一些不重要的东西.
 类型推断是这些函数中最重要的功能之一,也就是,你不需要指定列的类型到底是数值,整数,布尔值,还是字符串.日期和其他自定义类型的处理需要多花点工夫才行.首先我们来看一个以逗号分隔的CSV文本文件:
 由于ex1.csv是以逗号分隔,索引我们可以使用read_csv将其读入一个DataFrame:

In [1]:
import json
import csv
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
df = pd.read_csv('./ex1.csv')
df


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


 我们也可以用read_table,只不过需要指定分隔符而已:

In [2]:
pd.read_table('./ex1.csv', sep=',')


/Users/mohenoo/Librarys/anaconda3/envs/python_36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


 并不是所有文件都有标题行.
 读入这种文件的办法有两个,你可以让pandas为其分配默认的列名,也可以自己定义列名:

In [3]:
pd.read_csv('./ex2.csv', header=None)


,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_csv('./ex2.csv', names=['a', 'b', 'c', 'd', 'message'])


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


 假设你希望将message列做成DataFrame的索引,你可以明确表示要将该列放到索引4的位置上,也可以通过index_col参数指定“message”:

In [5]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./ex2.csv', names=names, index_col='message')


,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


 如果希望将多个列做成一个层次化索引,只需要传入由列编号或列名组成的列表即可:

In [6]:
parsed = pd.read_csv("./csv_mindex.csv", index_col=['key1', 'key2'])
parsed


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

 有些表格可能不是用固定的分隔符去分隔字段的.对于这种情况,可以编写一个正则表达式来作为read_table的分隔符.ex3.txt各个字段由数量不定的空白符分隔,虽然可以对其做一些手工调整,但这个情况还是用正则表达式\s+表示

In [7]:
result = pd.read_table('./ex3.txt', sep='\s+')
result


/Users/mohenoo/Librarys/anaconda3/envs/python_36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,,[
A,B,"C,"
"aaa-0.264438-1.026059-0.6195000,",NaN,NaN
bbb,0.927272,"0.302904-0.032399,"
"ccc-0.264273-0.386314-0.217601,",NaN,NaN
ddd-0.871858-0.348382,1.100491,NaN
],NaN,NaN


 这里,由于列名比数据行的数量少,所以read_table推断第一列应该是DataFrame的索引.
 这些解析器函数还有许多参数,可以帮助你处理各种各样的异形文件格式.
 缺失值处理是文本解析任务中的一个重要组成部分.缺失数据经常是要么没有,要么用某个标记表示.默认情况下,pandas会用一组经常出现的标记值进行识别,如NA,-1.IND以及NULL等:

In [8]:
result = pd.read_csv('./ex5.csv')
result


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [9]:
pd.isnull(result)


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


 na_values可以接受一组用于表示缺失值的字符串

In [10]:
result = pd.read_csv('./ex5.csv', na_values=['NULL'])
result


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


 可以用一个字典为各列指定不同的NA标记值:

In [11]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('./ex5.csv', na_values=sentinels)


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


 ### 逐块读取文本文件
 在处理很大的文件时,或找出大文件中的参数集以便于后续处理时,你可能只想读取文件的一小部分或逐块对文件进行迭代.
 如果只想读取几行(避免读取整个文件),通过nrows进行指定即可.

In [12]:
pd.read_csv('./ex5.csv', nrows=1)


,something,a,b,c,d,message
0,one,1,2,3,4,NaN


 要逐块读取文件,需要设置chunksize(行数):

In [13]:
chunker = pd.read_csv('./ex5.csv', chunksize=3)
chunker


 read_csv所返回的这个TextPaeser对象使你可以根据chunksize对文件进行逐块迭代.比如,我们可以将值计数聚合到“key”列中:

In [14]:
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:10]


KeyError: 'key'

 TextParser还有一个get_chunk方法,它使你可以读取任意大小的块.
 ### 将数据写出到文本格式
 数据也可以被输出为分隔符格式的文本.

In [15]:
data = pd.read_csv('./ex5.csv')
data


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


 利用率DataFrame的to_csv方法,我们可以将数据写到一个以逗号分隔的文件:

In [16]:
data.to_csv('./out.csv')


In [17]:
!cat ./out.csv


,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


 当然,还可以使用其他分隔符(由于这里直接写出到sys.stdout,所以仅仅是打印出文本结果而已):

In [18]:
data.to_csv(sys.stdout, sep='|')


|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


 缺失值在输出结果中会被表示为空字符串,你可能希望将其表示为别的标记值:

In [19]:
data.to_csv(sys.stdout, na_rep='NULL')


,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


 如果没有设置其他选项,则会写出行和列的标签.当然它们都可以被禁用:

In [20]:
data.to_csv(sys.stdout, index=False, header=False)


one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


 此外,你还可以只写出一部分的列,并以你指定的顺序排列:

In [21]:
data.to_csv(sys.stdout, index=False, columns=list('abc'))


a,b,c
1,2,3.0
5,6,
9,10,11.0


 Series也有一个to_csv方法:

In [22]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('tseries.csv')
!cat tseries.csv


2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


/Users/mohenoo/Librarys/anaconda3/envs/python_36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


 虽然只需一点整理工作(无header行,第一列作索引)就能用read_csv将CSV文件读取为Series,但还有一个更为方便的from_csv方法:

In [23]:
Series.from_csv('./tseries.csv', parse_dates=True)


/Users/mohenoo/Librarys/anaconda3/envs/python_36/lib/python3.6/site-packages/pandas/core/series.py:4137: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

 ### 手工处理分隔符格式
 大部分存储在磁盘上的表格型数据都能用pandas.read_table进行加载.然而,有时还是需要做一些手工处理.由于接收到含有畸形行的文件而使read_table出毛病的情况并不少见.

In [24]:
!cat ex7.csv


"a","b","c"
"1","2","3"
"1","2","3","4"

 对于任何单字符分隔文件,可以直接使用python内置的csv模块.将任意已打开的文件或文件类型的对象传给csv.reader:

In [25]:
f = open("./ex7.csv")
reader = csv.reader(f)


 对这个reader进行迭代将会为每行产生一个元组(并移除了所有的引号):

In [26]:
for line in reader:
    print(line)


['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


 现在,为了使数据格式合乎要求,你需要对其做一些整理工作:

In [27]:
lines = list(csv.reader(open('./ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict


{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

 csv文件的形式有很多.只需定义csv.Dialect的一个子类即可定义出新格式(如专门的分隔符,字符串引用约定,行结束等):

In [28]:


class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = 1


reader = csv.reader(f, dialect=my_dialect)


 各个CSV语支的参数也可以以关键字的形式提供给csv.reader,而无需定义子类:
 不过,对于那些使用复杂分隔符或多字符分隔符的文件,csv模块就无能为力了.这种情况下,你就只能使用字符串的split方法或正则表达式方法re.split进行行拆分和其他整理工作了.
 要手工输出分隔符文件,你可以使用csv.writer.它接受一个已打开且可写的文件对象以及跟csv.reader相同的那些语支和格式化选项.

In [29]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))


 ### JSON数据
 JSON是一种比表格型文本格式(如CSV)灵活得多的数据格式.除其空值null和一些其他的细微差别(如列表末尾不允许存在多余的逗号)之外,JSON非常接近于有效的Python代码.基本类型有对象(字典),数组(列表),字符串,数值,布尔型以及null.对象中所有的键都必须是字符串.许多Python库都可以读写JSON数据:

In [30]:
obj = """
{
 "name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"slblings": [
    {"name":"Scott", "age": 25, "pet": "Zuko"},
    {"name":"Katie","age":33,"pet":"Cisco"}]   
}
"""
result = json.loads(obj)
result



{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'slblings': [{'name': 'Scott', 'age': 25, 'pet': 'Zuko'},
  {'name': 'Katie', 'age': 33, 'pet': 'Cisco'}]}

 相反,json.dumpa则将Python对象转换成JSON格式:
 如何将(一个或一组)JSON对象转换为DataFrame或其他便于分析的数据结构就由你决定了.最简单方便的方式是:向DataFrame构造器传入一组JSON对象,并选区数据字段的子集.

In [31]:
sliblings = DataFrame(result['slblings'], columns=['name', 'age'])
sliblings


,name,age
0,Scott,25
1,Katie,33


 ### XML和HTML:Web信息收集
 Python有许多读写HTML和XML格式数据的库.lxml就是其中一个,它能够高效且可靠地解析大文件.lxml有多个编程接口.
 ## 二进制数据格式
 实现数据的二进制格式存储最简单的办法之一是使用Python内置的pickle序列化.为了使用方便,pandas对象都有一个用于将数据以pickle形式保存到磁盘上的to_pickle:

In [32]:
frame = pd.read_csv('./ex1.csv')
frame


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [33]:
frame.to_pickle('frame_pickle')


 你可以通过另一个也很好用的pickle函数pandas.read_pickle:

In [34]:
pd.read_pickle('./frame_pickle')


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


 **警告:** pickle仅建议用于短期存储格式.其原因是很难保证该格式永远是稳定的.
 ### 使用HDF5格式
 很多工具都能实现高效读写磁盘上以二进制格式存储的科学数据.HDF5就是其中一个流行的工业级库,它是一个C库,带有许多语言的接口.HDF5中的HDF指的是层次型数据格式.每个HDF5文件都含有一个文件系统式的节点结构,它使你能够存储多个数据集并支持元数据.与其他简单格式相比,HDF5支持多种压缩器的即时压缩,还能更高效地存储重复模式数据.
 Python中的HDF5库有两个接口(即PyTables和h5py),它们各自采取了不同的问题解决方式.h5py提供了一种直接而高级的HDF5API访问接口,而PyTables则抽象了HDF5的许多细节以提供多种灵活的数据容器,表索引,查询功能以及对核外计算技术的某些支持.
 ### 读取Microsoft Excel文件
 pandas的ExcelFile类支持读取存储在Excel中的表格型数据.
 ### 使用HTML和Web API
 许多网站都有一些通过JSON或其他格式提供数据的公共API.通过Python访问这些API的办法有很多.一个简单的办法是用requests包.
 ### 使用数据库
 在许多应用中,数据很少取自文本文件,因为用这种 方式存储大量数据很低效.基于SQL的关系型数据库使用非常广泛,此外还有一些非SQL型数据库也变得非常流行.
 将数据从SQL加载到DataFrame的过程很简单.连接数据库后,从表中选取数据时,大部分Python SQL驱动器都会返回一个元组列表.pandas有一个可以简化读取的函数read_frame函数.
 ### 存储MongoDB中的数据
 NoSQL数据库有许多不同的形式.有些是简单的字典式键值对存储,另一些则是基于文档的.